# 스토리지 벤치마크

이 노트북에서는 다음 저장소 형식을 비교해 보겠습니다.- CSV: 쉼표로 구분된 표준 일반 텍스트 파일 형식입니다.- HDF5: 국립 슈퍼컴퓨팅 애플리케이션 센터에서 처음 개발된 계층적 데이터 형식입니다. 이는 PyTables 라이브러리를 사용하여 Pandas에서 사용할 수 있는 숫자 데이터를 위한 빠르고 확장 가능한 저장 형식입니다.- Parquet: 효율적인 데이터 압축 및 인코딩을 제공하고 Cloudera와 Twitter에서 개발한 바이너리, 열 형식 스토리지 형식인 Apache Hadoop 에코시스템의 일부입니다. 팬더의 원저자인 Wes McKinney가 이끄는 `pyarrow` 라이브러리를 통해 팬더에 사용할 수 있습니다.
이 노트북은 숫자나 텍스트 데이터 또는 둘 다를 포함하도록 구성할 수 있는 테스트 DataFrame을 사용하여 이전 라이브러리의 성능을 비교합니다. HDF5 라이브러리의 경우 고정 형식과 테이블 형식을 모두 테스트합니다. 테이블 형식은 쿼리를 허용하고 추가될 수 있습니다.
## 용법
책에 사용된 차트를 다시 생성하려면 다음과 같이 `data_type`에 대한 다른 설정과 `generate_test_data`에 대한 인수를 사용하여 '결과 저장' 섹션까지 이 노트북을 두 번 실행해야 합니다.1. `data_type='Numeric`: `numerical_cols=2000`, `text_cols=0`(기본값)2. __자리 표시자_0__: __자리 표시자_1__, __자리 표시자_2__

## 가져오기 및 설정

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import string

In [3]:
sns.set_style('whitegrid')

In [4]:
results = {}

## 테스트 데이터 생성

숫자나 텍스트 데이터 또는 둘 다를 포함하도록 구성할 수 있는 `DataFrame` 테스트입니다. HDF5 라이브러리의 경우 고정 형식과 테이블 형식을 모두 테스트합니다.

In [5]:
def generate_test_data(nrows=100000, numerical_cols=2000, text_cols=0, text_length=10):
    s = "".join([random.choice(string.ascii_letters)
                 for _ in range(text_length)])
    data = pd.concat([pd.DataFrame(np.random.random(size=(nrows, numerical_cols))),
                      pd.DataFrame(np.full(shape=(nrows, text_cols), fill_value=s))],
                     axis=1, ignore_index=True)
    data.columns = [str(i) for i in data.columns]
    return data

In [6]:
data_type = 'Numeric'

In [7]:
df = generate_test_data(numerical_cols=1000, text_cols=1000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 2000 entries, 0 to 1999
dtypes: float64(1000), object(1000)
memory usage: 1.5+ GB


## 쪽매 세공

### 크기

In [8]:
parquet_file = Path('test.parquet')

In [9]:
df.to_parquet(parquet_file)
size = parquet_file.stat().st_size

### 읽다

In [10]:
%%timeit -o
df = pd.read_parquet(parquet_file)

4.86 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 4.86 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [11]:
read = _

In [12]:
parquet_file.unlink()

### 쓰다

In [13]:
%%timeit -o
df.to_parquet(parquet_file)
parquet_file.unlink()

43.5 s ± 1.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 43.5 s ± 1.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [14]:
write = _

### 결과

In [15]:
results['Parquet'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

## HDF5

In [16]:
test_store = Path('index.h5')

### 고정 형식

#### 크기

In [17]:
with pd.HDFStore(test_store) as store:
    store.put('file', df)
size = test_store.stat().st_size

#### 읽다

In [18]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    store.get('file')

2min 7s ± 2.73 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 2min 7s ± 2.73 s per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [19]:
read = _

In [20]:
test_store.unlink()

#### 쓰다

In [21]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    store.put('file', df)
test_store.unlink()

1min 10s ± 1.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 1min 10s ± 1.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [22]:
write = _

#### 결과

In [23]:
results['HDF Fixed'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

### 테이블 형식

#### 크기

In [24]:
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t')
size = test_store.stat().st_size    

#### 읽다

In [ ]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    df = store.get('file')

In [ ]:
read = _

In [ ]:
test_store.unlink()

#### 쓰다

테이블 형식의 `write`은 텍스트 데이터에서 작동하지 않습니다.

In [ ]:
%%timeit -o
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t')
test_store.unlink()    

In [ ]:
write = _

#### 결과

In [ ]:
results['HDF Table'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

### 테이블 선택

#### 크기

In [ ]:
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t', data_columns=['company', 'form'])
size = test_store.stat().st_size 

#### 읽다

In [ ]:
company = 'APPLE INC'

In [ ]:
%%timeit
with pd.HDFStore(test_store) as store:
    s = store.get('file')

In [ ]:
read = _

In [ ]:
test_store.unlink()

#### 쓰다

In [ ]:
%%timeit
with pd.HDFStore(test_store) as store:
    store.append('file', df, format='t', data_columns=['company', 'form'])
test_store.unlink() 

In [ ]:
write = _

#### 결과

In [ ]:
results['HDF Select'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

## CSV

In [ ]:
test_csv = Path('test.csv')

### 크기

In [ ]:
df.to_csv(test_csv)
test_csv.stat().st_size

### 읽다

In [ ]:
%%timeit -o
df = pd.read_csv(test_csv)

In [ ]:
read = _

In [ ]:
test_csv.unlink()  

### 쓰다

In [ ]:
%%timeit -o
df.to_csv(test_csv)
test_csv.unlink()

In [ ]:
write = _

### 결과

In [ ]:
results['CSV'] = {'read': np.mean(read.all_runs), 'write': np.mean(write.all_runs), 'size': size}

## 결과 저장

In [ ]:
pd.DataFrame(results).assign(Data=data_type).to_csv(f'{data_type}.csv')

## 결과 표시

위에서 설명한 대로 `Usage`에서 노트북을 두 번 실행하여 서로 다른 테스트 데이터에 대한 결과가 포함된 두 개의 `csv` 파일을 생성하세요.

In [ ]:
df = (pd.read_csv('Numeric.csv', index_col=0)
      .append(pd.read_csv('Mixed.csv', index_col=0))
      .rename(columns=str.capitalize))
df.index.name='Storage'
df = df.set_index('Data', append=True).unstack()
df.Size /= 1e9

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(16, 4))
for i, op in enumerate(['Read', 'Write', 'Size']):
    flag= op in ['Read', 'Write']
    df.loc[:, op].plot.barh(title=op, ax=axes[i], logx=flag)
    if flag:
        axes[i].set_xlabel('seconds (log scale)')
    else:
        axes[i].set_xlabel('GB')
fig.tight_layout()
fig.savefig('storage', dpi=300);